In [61]:
! pip install tensorflow numpy matplotlib pandas scikit-learn opencv-python ipywidgets torch

import numpy as np
import matplotlib.pyplot as plt
import os
from glob import glob
import tensorflow as tf
import torch
import math
import time
from IPython.display import clear_output, display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from ipywidgets import interact, FloatSlider, IntSlider, Checkbox
from IPython.display import display, clear_output
import cv2

In [63]:
# all the stuff to calculate the transformation matrices

def rotation_x(theta):
    """Pitch: Rotation about the X-axis"""
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[1, 0,  0],
                     [0, c, -s],
                     [0, s,  c]])

def rotation_y(theta):
    """Roll: Rotation about the Y-axis"""
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[ c, 0, s],
                     [ 0, 1, 0],
                     [-s, 0, c]])

def rotation_z(theta):
    """Yaw: Rotation about the Z-axis"""
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[c, -s, 0],
                     [s,  c, 0],
                     [0,  0, 1]])


def getDomFace(yaw, pitch):
    if pitch > 35:
        face='top'
    elif 45 < yaw <= 135:
        face='passenger'
    elif 135 < yaw <= 225:
        face='front'
    elif 225 < yaw <= 315:
        face='driver'
    else:
        face='back'
    
    return face

facePrePitches = {
    "back": 5,
    "front": 20, 
}


def yawDiffCalc(domFace, yaw):
    if domFace == "front":
        return yaw - 180
    elif domFace == "back":
        if yaw > 180:
            return yaw - 360
        else:
            return yaw
    elif domFace == "passenger":
        return yaw - 90
    elif domFace == "driver":
        return yaw - 270
    else:
        return yaw


def calc3dRotMat(pitch, yaw):
    domFace = getDomFace(yaw, pitch)
    yawDiff = yawDiffCalc(domFace, yaw)

    
    yaw_rad = np.radians(yaw)
    

    res_mat = np.eye(3)

    if domFace=='top':
        #yaw by yaw
        top_mat = rotation_z(yaw_rad)
        yawDiff = 0

        res_mat = top_mat @ res_mat

        
    
    elif domFace in ["front", "back"]:
        #yaw by 90
        front_back_yaw_mat = rotation_z(np.radians(90))

        #pitch by facePrePitch
        pre_pitch = np.radians(facePrePitches[domFace])
        pre_pitch_mat = rotation_x(pre_pitch)

        res_mat = pre_pitch_mat @ front_back_yaw_mat @ res_mat


    #roll by yawDiff
    yawDiff_rad = np.radians(yawDiff)
    roll_mat = rotation_y(yawDiff_rad)

    #pitch by pitch
    pitch_rad = np.radians(pitch)
    pitch_mat = rotation_x(pitch_rad)

    res_mat = pitch_mat @ roll_mat @ res_mat

    return res_mat



       

In [64]:

def transform_projection(image, pitch, yaw, distance, shift=(0,0)):

    h, w = image.shape[:2]

    shift_x, shift_y = shift

    scale = 15 - distance

    # ---- Build Projection Matrix ----
    f = 500
    cx, cy = w//2, h//2


    # Define corner points in 3D
    corners_3d = np.array([
        [-w/2, -h/2, 0],
        [ w/2, -h/2, 0],
        [ w/2,  h/2, 0],
        [-w/2,  h/2, 0]
    ])

    # Apply rotation
    R = calc3dRotMat(pitch, yaw)
    rotated = corners_3d @ R.T

    # Apply scaling
    rotated *= scale

    # Project back to 2D
    projected = rotated.copy()
    projected[:,0] = f * projected[:,0] / (f + projected[:,2]) + cx + shift_x
    projected[:,1] = f * projected[:,1] / (f + projected[:,2]) + cy + shift_y

    # Warp perspective
    src_pts = np.array([[0,0],[w,0],[w,h],[0,h]], dtype=np.float32)
    dst_pts = projected[:,:2].astype(np.float32)
    M = cv2.getPerspectiveTransform(src_pts, dst_pts)

    output = cv2.warpPerspective(image, M, (w+200, h+200))
    return output

